# ORF: Open Reanding Frames

### Problem  
https://rosalind.info/problems/orf/

Either strand of a DNA double helix can serve as the coding strand for RNA transcription. Hence, a given DNA string implies six total reading frames, or ways in which the same region of DNA can be translated into amino acids: three reading frames result from reading the string itself, whereas three more result from reading its reverse complement.

An open reading frame (ORF) is one which starts from the start codon and ends by stop codon, without any other stop codons in between. Thus, a candidate protein string is derived by translating an open reading frame into amino acids until a stop codon is reached.

**Given:** A DNA string s of length at most 1 kbp in FASTA format.

**Return:** Every distinct candidate protein string that can be translated from ORFs of s. Strings can be returned in any order.

## Level 1: Solve Rosalind problem ORF

#### Strategy:
Not going for the most straight-forward path because I want to create useful intermediate steps and I want to retain frame information.
1. Create dictionary with codons of all 6 possible frames (sense_1, sense_2, sense_3, asense_1, asense_2, asense_3).
2. Find all start and stop codon positions in frames dictionary and store positions in new dictionary (still organized by frame).
3. Select all the ORF's (i.e. codons from start to stop w/o intermediate stop codons), translate into peptide sequence and add peptide to new dictionary organized by frame.
4. Print all the unique possible peptides.

NOTE:
Because I want to also practice bipython, I will create an alternative version using it whenever it is useful.

In [15]:
codon_table = {'UUU':'F', 'UUC':'F', 'UUA':'L', 'UUG':'L', 'UCU':'S',
               'UCC':'S', 'UCA':'S', 'UCG':'S', 'UAU':'Y', 'UAC':'Y',
               'UAA':'stop', 'UAG':'stop', 'UGU':'C', 'UGC':'C', 'UGA':'stop',
               'UGG':'W', 'CUU':'L', 'CUC':'L', 'CUA':'L', 'CUG':'L',
               'CCU':'P', 'CCC':'P', 'CCA':'P', 'CCG':'P', 'CAU':'H',
               'CAC':'H', 'CAA':'Q', 'CAG':'Q', 'CGU':'R', 'CGC':'R',
               'CGA':'R', 'CGG':'R', 'AUU':'I', 'AUC':'I', 'AUA':'I',
               'AUG':'M', 'ACU':'T', 'ACC':'T', 'ACA':'T', 'ACG':'T',
               'AAU':'N', 'AAC':'N', 'AAA':'K', 'AAG':'K', 'AGU':'S',
               'AGC':'S', 'AGA':'R', 'AGG':'R', 'GUU':'V', 'GUC':'V',
               'GUA':'V', 'GUG':'V', 'GCU':'A', 'GCC':'A', 'GCA':'A',
               'GCG':'A', 'GAU':'D', 'GAC':'D', 'GAA':'E', 'GAG':'E',
               'GGU':'G', 'GGC':'G', 'GGA':'G', 'GGG':'G'}

with open('rosalind_orf.txt','r') as file:
    infile = file.read().split('\n')
    DNA = ''.join(infile[1:])
    RNA_sense = DNA.replace('T','U')
    RNA_asense = RNA_sense.replace('A','u').replace('U','a').replace('C','g').replace('G','c').upper()[::-1]
    RNA= {'RNA_sense' : RNA_sense,
          'RNA_asense' : RNA_asense}


# 1. Create dictionary with codons of all 6 possible frames:

frames = {}
for strand, seq in RNA.items():
    for frame_start in range(3):
        key = f'{strand}_{frame_start+1}'
        frames[key] = [seq[i:i+3] for i in range(frame_start,len(seq)-2, 3)]


# 2. Find all start and stop codon positions in frames dictionary and store them in new dictionaries.
# Note: could have also stored in list, but I wanted to retain the frame information for myself

start_codon = 'AUG'
stop_codon = {'UAA','UAG','UGA'}


start_i = {}
stop_i = {}

for frame, codons in frames.items():
    start_i[frame]= [i for i, codon in enumerate(codons) if codon == start_codon]
    stop_i[frame]= [i for i, codon in enumerate(codons) if codon in stop_codon]


# 3. Select all ORF's (start to stop codons w/o an intermediate stop codon), translate into peptide and organize by frame in new dictionary

ORFs = {}
for frame, codons in frames.items():
    starts = start_i[frame]
    stops = stop_i[frame]

    for start in starts:
        for stop in stops:
            if stop > start:
                mid_stops = [mid_codon for mid_codon in stops if start < mid_codon < stop] # Find intermediate stop codons
                if not mid_stops: # Proceed if there aren't intermediate stop codons
                    orf_codons = codons[start:stop]
                    peptide = ''.join(codon_table[codon] for codon in orf_codons)
                    if frame not in ORFs:
                        ORFs[frame] = []
                    ORFs[frame].append(peptide)
                    break


# 4. Print all the unique possible peptides

unique_peptides = set() # Kind of like a list but for unique values and unordered
for peptide_list in ORFs.values():
    unique_peptides.update(peptide_list)
for unique_peptide in unique_peptides:
    print(unique_peptide)

MVNRILDRLY
MDCQDYLRLWSRLATGADFSTWNSLLSAAVVILQTSSIDKLPIYFYPRFL
MISSIAKSSGFAIVRNRSCYSSKLSSSHISLG
M
MSPVL
MGKQASFEGAIQRGWPHDTGS
MRQTNLT
MGKQASSFDSKLLDCLLDESGPIVGFTGTLDRNRLAAYLSN
MSIQRPVS
MTRGHETGR
MLMEVTLGAALPTSVGSRIRTRLVISYSVGSLPSSCT
MKRGAECSWKSL
MTSRVLIRDPTEVGSAAPRVTSMSIQRPVS
MRPTSLNRALE
MDDNSRCN
MTPVFP
MPLLRHT
MFFHRGML
MDT
MSHTVIRGFGHSDSSRPQGALGAGGLSSVLAPACASMPLLRHT
ML
MTKTSYYSVRHHNIPRWKNMDDNSRCN
MPYGLPRLFTIVVPACYGGGLLHVEQSPLCCCCNLAN
MTLSIAGQGFG
MTPCHAANLAESRPRMTPVFP
MMSHTVIRGFGHSDSSRPQGALGAGGLSSVLAPACASMPLLRHT
MEVTLGAALPTSVGSRIRTRLVISYSVGSLPSSCT


##### Biopython variant

# Leveling up

I want to expand on this Rosalind problem by tweaking the goals into incremental levels of complexity - each level introducing new skills to learn and build on.

- Level 1: Solve the problem
- Level 2: Analyse putative ORF data and generate fasta files
- Level 3: Scale to analysis of multiple sequences and run through bash
- Level 4: Interface


## Level 2: Analyse putative ORF data and generate fasta files

**Create program to assemble and analyse data on the putative ORFs and peptides for a DNA sequence.**

This means generating a data frame including:
- Unique putative ORF and peptide ID; Frame and Strand; Start and stop positions; ORF length and peptide size; Overlap information
And plotting some data:
- Size x position plot; Overlaps

Additionally, program should be interactive and ask whether you want to generate fasta files containing the putative ORFs (in DNA and/or RNA) and/or the respective peptides, along with what sequence identifying information you would like to include.

Rather than just returning a list of all the putative peptides translated from a DNA string, I want to create a program to assemble a data frame storing more useful information about the putative ORF's of a DNA sequence. The resulting data frame would include:
- ID (generate an ID for each putative ORF: dnaID_orf_1, dnaID_orf_2...)
- Frame (Sense +1, Sense +2, Sense +3, Antisense +1, Antisense +2, Antisense +3)
- Start (position of first nucleotide in ORF)
- Stop (position of last nucleotide in ORF)
- Stop codon sequence (which of the three stop codons?)
- ORF length (nucleotides in putative ORF)
- Peptide size (number of aminoacids in putative peptide)
- Overlap (does ORF overlap with another ORF?)

Learning goals:
- Intro do Pandas data analysis
    - Creating data-frames
    - Plotting data
- Writing fasta files
- Creating interactive functions

Steps:
- [x] Use Pandas to create data-frame
- [ ] Plot data
- [ ] Write fasta files (DNA, RNA and protein)
- [ ] Integrate code into interactive function

#### 2.1 Use Pandas to create data-frame

In [18]:

# Step 1: Prepare sequence for analysis
with open('rosalind_orf.txt', 'r') as file:
    infile = file.read().splitlines()
    id = infile[0].replace('>','')
    dna_seq = ''.join(infile[1:])
    print(len(dna_seq))
    rna_seq_sense = dna_seq.replace('T','U')
    rna__seq_asense = rna_seq_sense.replace('A','u').replace('U','a').replace('C','g').replace('G','c').upper()[::-1]
    rna_seq= {'Sense' : rna_seq_sense, 'Antisense' : rna__seq_asense}


# Step 2: Set start and stop codons
start_codon = ['AUG']
stop_codon = ['UAA','UAG','UGA']


# Step 3: Create each data variable as a list
start = []
stop = []
strand = []
frame = []
length = []
peptide_size = []
overlap = []


# Step 4: Fill-up the data lists
for dir, seq in rna_seq.items():
    for i in range(len(seq)-5): # -5 because I only want to find start codons that have enough subsequent sequence for a stop codon
        if seq[i:i+3] in start_codon: # if you find a start codon...
            for j in range(i+3 ,len(seq)-2, 3):
                if seq[j:j+3] in stop_codon: # ...then find a stop codon, fill-out the variables: 
                    if dir == 'Sense':
                        start.append(i)
                        stop.append(j)
                        strand.append(dir)
                        if (i+3) % 3 == 0:
                            frame.append(f'{dir} +1')
                        elif (i+2) % 3 == 0:
                            frame.append(f'{dir} +2')
                        elif (i+1) % 3 == 0:
                            frame.append(f'{dir} +3')
                        length.append(j-i)
                        peptide_size.append((j-i)//3) # // needed to get integer instead of float?
                        break
                    elif dir == 'Antisense':
                        start.append(len(seq)-i)
                        stop.append(len(seq)-j)
                        strand.append(dir)
                        if (i+3) % 3 == 0:
                            frame.append(f'{dir} +1')
                        elif (i+2) % 3 == 0:
                            frame.append(f'{dir} +2')
                        elif (i+1) % 3 == 0:
                            frame.append(f'{dir} +3')
                        length.append(j-i)
                        peptide_size.append((j-i)//3)
                        break


orf_id = []
for i in range(len(start)):
    orf_id.append(f'{id}_orf_{i+1}')

# This is kind of wrong because of the two directions.
# So I won't be including it in the data fram until its fixed
for k in range(len(orf_id)):
    overlaps = []
    for l in range(len(orf_id)):
        if k != l:
            if start[k] < stop[l] and stop[k] > start[l]:
                overlaps.append(orf_id[k])
    if overlaps:
        overlap.append(', '.join(overlaps))
    else:
        overlap.append('None')


# Step 5: Assemble data variable lists into a dictionary, then into a data-frame using pandas
orf_data = {
    'ID' : orf_id,
    'Start' : start,
    'Stop' : stop,
    'Frame' : frame,
    'Strand' : strand,
    'Length' : length,
    'Peptide size (aa)' : peptide_size
    #'Overlap': overlap
}

import pandas as pd
orf_df = pd.DataFrame(orf_data) # turn dictionary into data-frame
orf_df.to_csv('orf_df.csv') # save in csv file

orf_df

# Missing:
# Write csv file with dataframe
# Write fasta file with peptides
# Fix overlap situation


#for index, row in df.iterrows():
#    if row['Strand'] == 'Sense':
#        print(rna_seq_sense[row['Start']:row['Stop']])
#    elif row['Strand'] == 'Antisense':
#        print(rna__seq_asense[row['Start']:row['Stop']])



# Use Pandas to make dataframe with variables above
# Then use it to select sense orfs and use start and stop to get DNA sequence, then to the same for antisense orfs
# Then transcribe, then translate
# Write fasta file for each sequence type


926


,ID,Start,Stop,Frame,Strand,Length,Peptide size (aa),Overlap
0,Rosalind_3903_orf_1,35,65,Sense +3,Sense,30,10,None
1,Rosalind_3903_orf_2,198,279,Sense +1,Sense,81,27,"Rosalind_3903_orf_2, Rosalind_3903_orf_2, Rosa..."
2,Rosalind_3903_orf_3,255,279,Sense +1,Sense,24,8,Rosalind_3903_orf_3
3,Rosalind_3903_orf_4,308,398,Sense +3,Sense,90,30,"Rosalind_3903_orf_4, Rosalind_3903_orf_4"
4,Rosalind_3903_orf_5,374,398,Sense +3,Sense,24,8,"Rosalind_3903_orf_5, Rosalind_3903_orf_5"
5,Rosalind_3903_orf_6,397,460,Sense +2,Sense,63,21,"Rosalind_3903_orf_6, Rosalind_3903_orf_6, Rosa..."
6,Rosalind_3903_orf_7,410,443,Sense +3,Sense,33,11,"Rosalind_3903_orf_7, Rosalind_3903_orf_7"
7,Rosalind_3903_orf_8,442,460,Sense +2,Sense,18,6,"Rosalind_3903_orf_8, Rosalind_3903_orf_8"
8,Rosalind_3903_orf_9,465,588,Sense +1,Sense,123,41,"Rosalind_3903_orf_9, Rosalind_3903_orf_9"
9,Rosalind_3903_orf_10,517,532,Sense +2,Sense,15,5,Rosalind_3903_orf_10


#### 2.2 Plot data

Maybe at some point I will try to do it in python, but for now I wanted to do it in R.
It would be a good way to warm up my R skills and get some analysis done before I dive into plotting with python.
In the next block, I will try to do that R plot. Remember to change to the R kernel for it to work!

In [1]:
# Open csv file
# Length of sequence is x axis
# Putative ORF_id is the categorical y axis
# Start and stop positions define size and position of bars plotted along the x axis
# ORF_id are grouped depending on whether they occur in sense or antisense strand and placed, respectively, above or bellow the x axis

library(tidyverse)
library(ggplot2)

ERROR: Error in library(tidyverse): there is no package called 'tidyverse'


In [1]:
# Plotting putative orf's on sequence

# I actually think I want to do this with R ggplot
#library(tidyverse)
library(ggplot2)

# Create a data frame with the categories, start, and stop positions
df <- data.frame(
  Category = c('A1', 'A2', 'B1', 'B2'),
  Start = c(100, 400, 150, 500),
  Stop = c(300, 700, 350, 800),
  Group = c('Above', 'Above', 'Below', 'Below')
)

# Calculate the length of each bar
df$Length <- df$Stop - df$Start

# Create a new column for the adjusted position based on the group
df$Position <- ifelse(df$Group == 'Above', df$Start, -df$Start)
df$Length <- ifelse(df$Group == 'Above', df$Length, -df$Length)

# Create the plot
myplot <- ggplot(df, aes(x = Length, y = Category, fill = Group)) +
  geom_bar(stat = "identity", aes(xmin = Position), width = 0.5) +
  scale_x_continuous(limits = c(-1000, 1000)) +
  scale_fill_manual(values = c('skyblue', 'lightgreen')) +
  labs(
    title = 'Horizontal Bars for Two Groups of Categories (Above and Below)',
    x = 'Position', y = 'Categories'
  ) +
  theme_minimal()

print(myplot)

ERROR: Error: package or namespace load failed for ‘ggplot2’ in loadNamespace(i, c(lib.loc, .libPaths()), versionCheck = vI[[i]]):
 namespace ‘scales’ 1.2.0 is being loaded, but >= 1.3.0 is required
